In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv("D://data/churn.csv")

In [ ]:
# We have already performed EDA on this dataset and there is no scope for data cleaning as well

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
round(df.describe(),2).T

In [ ]:
df.head()

In [ ]:
df['Exited'].value_counts()

In [ ]:
# Check the class distribution
sns.countplot(df['Exited'])

In [ ]:
# Baseline accuracy
7963/10000

In [ ]:
df.columns

In [ ]:
dummies = pd.get_dummies(df[['Geography','Gender']], drop_first=True)

In [ ]:
dummies

In [ ]:
x = df.iloc[:,[2,5,6,7,8,9,10,11]]

In [ ]:
x = pd.concat([x,dummies], axis=1)

In [ ]:
y = df['Exited']

In [ ]:
x.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [ ]:
x_train.shape

In [ ]:
y_train.value_counts()

In [ ]:
6346/8000

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
model_lr = lr.fit(x_train,y_train)

In [ ]:
model_lr.coef_

In [ ]:
pred_prob=pd.Series(model_lr.predict_proba(x_test)[:,1])

In [ ]:
from sklearn import metrics

In [ ]:
# Look for the fitness of the model before using it for prediction using ROCR/ AUC

# Receiver Operating Characteristic Plot
# Area Under the Curve

ypred = model_lr.predict_proba(x_test)[:,1]
fpr,tpr, _ = metrics.roc_curve(y_test,ypred)
auc = metrics.roc_auc_score(y_test,ypred)
plt.plot(fpr,tpr,label = "Curve, Auc = "+str(auc))
plt.legend(loc=4)
plt.show()

In [ ]:
pred_churn = model_lr.predict(x_test)

In [ ]:
pred_churn

In [ ]:
pred_churn[:80]

In [ ]:
y_test[:40]

In [ ]:
pd.crosstab(y_test,pred_churn)

In [ ]:
pred_prob.shape

In [ ]:
pred_prob

In [ ]:
pd.crosstab(y_test.reset_index(drop=True),pred_prob>0.5)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [ ]:
confusion_matrix(y_test,pred_churn)

In [ ]:
(1584+15)/2000

In [ ]:
# Precision 

In [ ]:
15/(15+33)

In [ ]:
from statsmodels import api as sm

In [ ]:
m = sm.GLM(y_train,x_train,family=sm.families.Binomial())

In [ ]:
m1 = m.fit()

In [ ]:
m1.summary()

In [ ]:
m1.fittedvalues()

In [ ]:
pred_churn1 = m1.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_churn1>0.5)

In [ ]:
accuracy_score(y_test,pred_churn1>0.5)

In [ ]:
metrics.precision_score(y_test,pred_churn1>0.5)

In [ ]:
recall = 55/(55+328)

In [ ]:
# Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtree = DecisionTreeClassifier()

In [ ]:
d1 = dtree.fit(x_train,y_train)

In [ ]:
pred_tree = d1.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_tree)

In [ ]:
(1403+175)/2000

In [ ]:
175/(175+214)

In [ ]:
dtree = DecisionTreeClassifier(max_leaf_nodes=10, max_depth=5, min_samples_leaf=5)

In [ ]:
d2 = dtree.fit(x_train,y_train)

In [ ]:
from dtreeplt import dtreeplt

In [ ]:
dtree_class = dtreeplt(
    model = d2, 
    feature_names=x_train.columns, 
    target_names=np.array(["neg","pos"]))

In [ ]:
dtree_class.view()

In [ ]:
pred_tree1 = d2.predict(x_test)

In [ ]:
pd.crosstab(y_test,pred_tree1)

In [ ]:
(1554+161)/2000

In [ ]:
161/(161+63)

In [ ]:
# Decision Tree pruning
# Grid search / Hyper parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score

In [ ]:
from sklearn import tree

In [ ]:
params = {'max_depth' : [2,4,6,8,10],'max_leaf_nodes' :[5,10], 'min_samples_leaf' : [2,5]}
clf = tree.DecisionTreeClassifier()
gcv = GridSearchCV(estimator=clf, param_grid=params)
gcv.fit(x_train,y_train)

In [ ]:
d2.get_params().keys()

In [ ]:
gcv.best_estimator_

In [ ]:
model_gcv = gcv.best_estimator_
model_gcv.fit(x_train,y_train)
pred_gcv = model_gcv.predict(x_test)

In [ ]:
accuracy_score(y_test,pred_gcv)

In [ ]:
path = d2.cost_complexity_pruning_path(x_train,y_train)

In [ ]:
alpha = path['ccp_alphas']

In [ ]:
alpha

In [ ]:
acc_train, acc_test = [],[]

for i in alpha:
    tree = DecisionTreeClassifier(ccp_alpha=i)
    tree.fit(x_train,y_train)
    y_train.pred = tree.predict(x_train)
    y_test.pred = tree.predict(x_test)
    
    acc_train.append(accuracy_score(y_train,y_train.pred))
    acc_test.append(accuracy_score(y_test,y_test.pred))
    acc_train.append(metrics.precision_score(y_train,y_train.pred))
    acc_test.append(metrics.precision_score(y_test,y_test.pred))

In [ ]:
acc_train

In [176]:
alpha

array([0.        , 0.0039947 , 0.00459475, 0.00522287, 0.00701547,
       0.00874131, 0.01302253, 0.01530457, 0.03868233])

In [ ]:
acc_test

In [177]:
model_ccp = DecisionTreeClassifier(ccp_alpha=0.00459475)

In [178]:
model_ccp = model_ccp.fit(x_train,y_train)

In [181]:
pred_ccp=model_ccp.predict(x_test)

In [182]:
pd.crosstab(y_test,pred_ccp)

col_0,0,1
Exited,,
0,1593,24
1,291,92


In [183]:
accuracy_score(y_test,pred_ccp)

0.8425

In [186]:
accuracy_score(y_test,pred_tree1)

0.8575

In [187]:
accuracy_score(y_test,pred_gcv)

0.8575

In [189]:
92/(92+24)

0.7931034482758621

In [190]:
pd.crosstab(y_test,pred_tree1)

col_0,0,1
Exited,,
0,1554,63
1,222,161


In [191]:
161/(161+63)

0.71875

In [193]:
from sklearn.ensemble import RandomForestClassifier

In [194]:
rf = RandomForestClassifier()

In [195]:
model_rf = rf.fit(x_train,y_train)

In [196]:
pred_rf = model_rf.predict(x_test)

In [197]:
pd.crosstab(y_test,pred_rf)

col_0,0,1
Exited,,
0,1555,62
1,223,160


In [198]:
(1555+160)/2000

0.8575

In [199]:
160/(160+62)

0.7207207207207207